In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import time

from math import sqrt
from numpy import loadtxt
from itertools import product
from tqdm import tqdm
from numpy import loadtxt

import gc
from sklearn import preprocessing 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from catboost import CatBoostClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
def reduce_mem_usage(df):
    """ 
    iterate through all the columns of a dataframe and 
    modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}' 
                     'MB').format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        print(str(col_type))
        if str(col_type) == "datetime64[ns]":
            continue
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <\
                  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <\
                   np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <\
                   np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <\
                   np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            elif str(col_type) != "Timestamp":
                if c_min > np.finfo(np.float16).min and c_max <\
                   np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max <\
                   np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}' 
                              'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) 
                                             / start_mem))
    
    return df
def df_info(df):
    print("----------Top-5- Record----------")
    print(df.head(5))
    print("-----------Information-----------")
    print(df.info())
    print("-----------Data Types-----------")
    print(df.dtypes)
    print("----------Missing value-----------")
    print(df.isnull().sum())
    print("----------Null value-----------")
    print(df.isna().sum())
    print("----------Shape of Data----------")
    print(df.shape)
    print("----------description of Data----------")
    print(df.describe())
    print("----------Uniques of Data----------")
    print(df.nunique())
    print("------------Columns in data---------")
    print(df.columns)

def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df
def model_performance_sc_plot( predictions, labels, title ):
    min_val = max(max(predictions), max(labels))
    max_val = min(min(predictions), min(labels))
    
    performance_df = pd.DataFrame({"Labels": labels})
    performance_df["Predictions"] = predictions
    sns.jointplot(y = "Labels", x = "Predictions", data = performance_df,kind = "reg")
    plt.plot([min_val,max_val],[min_val, max_val],"m--")
    plt.title(title)
    plt.show()
    
def calculate_counts(column_name, ref_sets, extension_set):
    ref_sets_ids = [set(ref[column_name]) for ref in ref_sets]
    ext_ids = set(extension_set[column_name])
    
    refs_union = reduce(lambda s1, s2: s1 | s2, ref_sets_ids)
    
    ref_counts = [len(ref) for ref in ref_sets_ids]
    ext_count = len(ext_ids)
    union_count = len(refs_union)
    intersection_count = len(ext_ids & refs_union)
    
    all_counts = ref_counts + [union_count, ext_count, intersection_count]
    res_index = ["Ref {}".format(i) for i in range(1, len(ref_sets) + 1)] +\
        ['Refs Union', 'Extension', 'Union x Extension']
    
    return pd.DataFrame({'Count': all_counts},
                        index=res_index)

In [ ]:
train_path = "/kaggle/input/airplane-accidents-severity-dataset/train.csv"
test_path = "/kaggle/input/airplane-accidents-severity-dataset/test.csv"
submission_path = "/kaggle/input/airplane-accidents-severity-dataset/sample_submission.csv"


In [ ]:
train = pd.read_csv(train_path)

In [ ]:
df_info(train)

In [ ]:
train.head()

In [ ]:
sns.set
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

In [ ]:
train["Severity"].replace("Minor_Damage_And_Injuries", 0 ,inplace= True)
train["Severity"].replace("Significant_Damage_And_Fatalities", 1 ,inplace= True)
train["Severity"].replace("Significant_Damage_And_Serious_Injuries", 2 ,inplace= True)
train["Severity"].replace("Highly_Fatal_And_Damaging", 3 ,inplace= True)

In [ ]:
train.head()

Accident type code mean wrt control metric, Severity, Turbulence, Maxelevation, Adverse Weatehr

In [ ]:
train["Control_Metric"] = train["Control_Metric"].clip(25,100)

In [ ]:
means_accident_code = train.groupby(["Accident_Type_Code"]).agg({"Severity": "mean","Safety_Score": "mean", "Days_Since_Inspection": "mean", "Total_Safety_Complaints" :"mean", "Control_Metric": "mean", "Turbulence_In_gforces" :"mean", "Cabin_Temperature": "mean", "Max_Elevation" :"mean","Violations": "mean", "Adverse_Weather_Metric" : "mean"})

In [ ]:
means_accident_code = means_accident_code.reset_index()

In [ ]:
cols = list(means_accident_code.columns)
for i in range(1,len(cols)):
    cols[i] = cols[i] + "_mean"
print(cols)
means_accident_code.columns = cols

In [ ]:
# train = train.merge(means_accident_code, on = ["Accident_Type_Code"], how = "left")

In [ ]:
train.groupby(["Accident_Type_Code"])["Accident_ID"].nunique()

In [ ]:
df = train.groupby(["Accident_Type_Code", "Severity"])["Severity"].agg({"no" : "count"})
mask = df.groupby(level=0).agg('idxmax')
df_count = df.loc[mask['no']]
df_count["no"] = df_count["no"]/ train.groupby(["Accident_Type_Code"])["Accident_ID"].nunique()
# df_count.drop(["no"], axis = 1, inplace= True)
df_count = df_count.reset_index()
df_count.columns = ["Accident_Type_Code", "Severity_max","Severity_count"]

In [ ]:
df_count

In [ ]:
train = train.merge(df_count, on = ["Accident_Type_Code"], how = "left")

In [ ]:
train.drop(["Accident_Type_Code"], axis = 1, inplace = True)

In [ ]:
train.head()

In [ ]:
X = train.drop(["Severity", "Accident_ID"], axis = 1)
Y = train["Severity"].values

In [ ]:
X.columns

In [ ]:
l_quan = ['Safety_Score','Control_Metric','Turbulence_In_gforces','Cabin_Temperature','Max_Elevation','Adverse_Weather_Metric','Total_Safety_Complaints','Severity_count']
l_cato = ["Violations" , "Days_Since_Inspection", "Severity_max"]

In [ ]:
for a in l_quan:
    sns.distplot(X[a])
    plt.show()

In [ ]:
for a in l_cato:
    sns.countplot(X[a])
    plt.show()

In [ ]:
for a in l_quan:
    x1,y1 = ecdf(X[a])
    x2,y2 = ecdf( np.random.normal( np.mean(X[a]), np.std(X[a]),size = 10000 ) )
    plt.plot(x1,y1,marker = "+", linestyle = None)
    plt.xlabel(a)
    plt.plot(x2,y2)
    plt.legend(["real", "theory"])
    plt.show()

In [ ]:
X["Total_Safety_Complaints"] = np.log(X["Total_Safety_Complaints"] + 1)

In [ ]:
a = "Total_Safety_Complaints"
x1,y1 = ecdf(X[a])
x2,y2 = ecdf( np.random.normal( np.mean(X[a]), np.std(X[a]),size = 10000 ) )
plt.plot(x1,y1,marker = "+", linestyle = None)
plt.xlabel(a)
plt.plot(x2,y2)
plt.legend(["real", "theory"])
plt.show()

log+1 ["Total_Safety_Complaints"]

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(X[l_quan])

In [ ]:
x_quan = pd.DataFrame(ss.transform(X[l_quan]),columns = l_quan, index = X.index )
X[l_quan] = x_quan
X.head()

In [ ]:
test = pd.read_csv(test_path)

In [ ]:
test.head().T

In [ ]:
# test = test.merge(means_accident_code, on = ["Accident_Type_Code"], how = "left")
test = test.merge(df_count, on = ["Accident_Type_Code"], how = "left")
test.drop(["Accident_Type_Code"], axis = 1, inplace = True)

In [ ]:
x_test = test.drop(["Accident_ID"], axis =1)

In [ ]:
x_test["Total_Safety_Complaints"] = np.log(x_test["Total_Safety_Complaints"] + 1)
x_test_quan = pd.DataFrame(ss.transform(x_test[l_quan]), columns = l_quan, index = x_test.index)
x_test[l_quan] = x_test_quan

In [ ]:
# corr = X.corr(train["Severity"])
# plt.figure(1,(50,16))
# ax = sns.heatmap(
#     corr, 
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# )
# ax.set_yticklabels(
#     ax.get_yticklabels(),
#     rotation=0,
#     horizontalalignment='right'
# )
# plt.show()

In [ ]:
# sns.countplot(Y)
# plt.show()

In [ ]:
# cb = CatBoostClassifier()
# from xgboost import XGBClassifier
# xgbRegressor = XGBClassifier()
# from sklearn.model_selection import RandomizedSearchCV


In [ ]:
# params_xgb = {
#     "num_rounds" : [100,200,400,],
#     "eta" : [0.05, 0.1,0.15, 0.2, 0.25, 0.30],
#     "learning_rate" : [0.28, 0.23, 0.25, 0.30],
#     "min_child_weight" : [1,3,5,7,10],
#     "gamma" : [0.05, 0.1,0.2, 0.3, 0.4],
#     "colsample_bytree" : [0.5,0.7, 0.8,0.9],
#     "subsample" : [0.8,0.9,0.5,0.7],
#     "max_depth" : [6,7,9,11,13],
#     "n_estimators" : [450, 350, 400, 550, 500],
# }

In [ ]:
# cb = CatBoostClassifier(iterations = 1500,
#                        max_ctr_complexity = 10,
#                        random_seed =  0,
#                        od_type = "Iter", 
#                        od_wait = 50,
#                        verbose = 100,
#                        depth  = 12,
#                        )

In [ ]:
# from sklearn.model_selection import KFold

# from collections import Counter
# n_splits = 5
# kf = KFold(n_splits = n_splits, shuffle = True)
# y_pred = np.zeros((Y.shape[0],4))
# test_pred_cb = np.zeros((test.shape[0], 4))
# sums_score = 0
# cat_features = l_cato
# for train_index, test_index in kf.split(X):
#     x_train, y_train = X.loc[train_index,:] , Y[train_index]
#     x_val, y_val = X.loc[test_index,:] , Y[test_index]
#     cb.fit(x_train , y_train ,cat_features = cat_features)
#     y_pred_cb = cb.predict_proba(x_val)
#     y_pred[test_index] += y_pred_cb/n_splits
#     test_pred_cb += cb.predict_proba(x_test)/n_splits
#     y_pred_cb = cb.predict(x_val)
#     print("F1_Score val1 : ", f1_score(y_val, y_pred_cb,average='weighted'))  
#     sums_score +=  f1_score(y_val, y_pred_cb,average='weighted')
# print("CV_mean: ", sums_score / n_splits)

In [ ]:
# xgbRegressor = XGBClassifier()

In [ ]:
# from sklearn.model_selection import KFold

# n_splits = 5
# kf = KFold(n_splits = n_splits, shuffle = True)
# y_pred_xg = np.zeros((Y.shape[0],4))
# sums_score = 0
# test_pred_xg = np.zeros((test.shape[0], 4))
# for train_index, test_index in kf.split(X):
#     x_train, y_train = X.loc[train_index,:] , Y[train_index]
#     x_val, y_val = X.loc[test_index,:] , Y[test_index]
    
   
#     xgbRegressor.fit(x_train , y_train, verbose = 100)
#     y_pred_x = xgbRegressor.predict_proba(x_val)
#     print(y_pred_x.shape)
#     y_pred_xg[test_index] += y_pred_x.reshape(-1,4)/n_splits
#     y_pred_x = xgbRegressor.predict(x_val)
#     test_pred_xg += xgbRegressor.predict_proba(x_test)/n_splits
#     print("F1_Score val1 : ", f1_score(y_train, xgbRegressor.predict(x_train),average='weighted'))
#     print("F1_Score val1 : ", f1_score(y_val, y_pred_x,average='weighted'))
#     sums_score +=  f1_score(y_val, y_pred_x,average='weighted')
# print("CV_mean: ", sums_score / n_splits)

In [ ]:
# def learning_rate_010_decay_power_099(current_iter):
#     base_learning_rate = 0.1
#     lr = base_learning_rate  * np.power(.99, current_iter)
#     return lr if lr > 1e-3 else 1e-3

# def learning_rate_010_decay_power_0995(current_iter):
#     base_learning_rate = 0.1
#     lr = base_learning_rate  * np.power(.995, current_iter)
#     return lr if lr > 1e-3 else 1e-3

# def learning_rate_005_decay_power_099(current_iter):
#     base_learning_rate = 0.05
#     lr = base_learning_rate  * np.power(.99, current_iter)
#     return lr if lr > 1e-3 else 1e-3

In [ ]:
# import lightgbm as lgb

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
# param_test ={
#     'boost': 'gbdt',
#     'learning_rate': 0.01,
#     "num_leaves" : 80,
#     "min_data_in_leaf" : 50,
#     "max_depth" : 13,
#     'min_sum_hessian_in_leaf': 10.0,
#     'num_threads': 8,
#     'tree_learner': 'serial',
#     'objective': 'multiclass', 
#     'verbosity': -1,
#     "num_classes" : 4
    
# }

In [ ]:
# from sklearn.model_selection import StratifiedKFold, KFold
# import lightgbm as lgb


In [ ]:
# from sklearn.model_selection import KFold
# num_folds = 5
# features = [c for c in train.columns if c not in ['Accident_ID', 'Severity']]

# folds = KFold(n_splits=num_folds, random_state=2319)
# oof = np.zeros((len(X),4))
# getVal = np.zeros((len(X), 4))
# predictions = np.zeros((len(test),4))
# feature_importance_df = pd.DataFrame()

In [ ]:
# y_pred_lgb = np.zeros((Y.shape[0],4))
# sums_score = 0
# for train_index, test_index in folds.split(X):
#     x_train, y_train = X.iloc[train_index][features] , Y[train_index]
#     x_val, y_val = X.iloc[test_index][features] , Y[test_index]
    
#     train_data = lgb.Dataset(x_train , label = y_train)
#     val_data = lgb.Dataset(x_val, label = y_val)
    
#     param = {
#     'boost': 'gbdt',
#     'learning_rate': 0.01,
#     "min_data_in_leaf" : 50,'num_leaves':100, 'objective':'multiclass', 'num_class':4, 'metric':'multi_logloss', 'seed':7}
#     num_round = 3000
    
#     clf = lgb.train(param, train_data, num_round, valid_sets=[val_data], early_stopping_rounds=120, verbose_eval=False, categorical_feature = l_cato)
#     oof[test_index] = clf.predict(train.iloc[test_index][features], num_iteration=clf.best_iteration)
#     getVal[test_index]+= clf.predict(train.iloc[test_index][features], num_iteration=clf.best_iteration) / folds.n_splits
    
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = features
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = sums_score + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#     sums_score+=1
#     predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
#     print("CV : ", f1_score(y_val, oof[test_index].argmax(axis = 1), average = "weighted"))


In [ ]:
# cols = (feature_importance_df[["feature", "importance"]]
#         .groupby("feature")
#         .mean()
#         .sort_values(by="importance", ascending=False)[:1000].index)
# best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

# plt.figure(figsize=(14,26))
# sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance",ascending=False))
# plt.title('LightGBM Features (averaged over folds)')
# plt.tight_layout()
# plt.savefig('lgbm_importances.png')

In [ ]:
# f_imps = {}
# for i in range(len(X.columns)):
#     f_imps[X.columns[i]] = xgbRegressor.feature_importances_[i]
# f_imps = {k: v for k, v in sorted(f_imps.items(), key=lambda item: item[1])}

In [ ]:
# list(f_imps.keys())

['Violations',
 'Max_Elevation',
 'Cabin_Temperature',
 'Total_Safety_Complaints',
 'Control_Metric_mean',
 'Cabin_Temperature_mean',
 'Turbulence_In_gforces_mean',
 'Adverse_Weather_Metric',
 'Turbulence_In_gforces',
 'Violations_mean',
 'Control_Metric',
 'Safety_Score',
 'Total_Safety_Complaints_mean',
 'Days_Since_Inspection',
 'Adverse_Weather_Metric_mean',
 'Severity_mean',
 'Max_Elevation_mean',
 'Safety_Score_mean',
 'Days_Since_Inspection_mean',
 'Severity_max']

In [ ]:
# plt.figure(1,(50,16))
# sns.barplot(x =list(f_imps.keys()), y = list(f_imps.values()), )
# plt.show()

In [ ]:
# first_level = pd.DataFrame(y_pred, columns = ["catboost_0","catboost_1","catboost_2","catboost_3"])
# # first_level[["XGB_0","XGB_1","XGB_2","XGB_3"]] =  y_pred_xg 
# first_level["XGB_0"] = y_pred_xg[:,0]
# first_level["XGB_1"] = y_pred_xg[:,1]
# first_level["XGB_2"] = y_pred_xg[:,2]
# first_level["XGB_3"] = y_pred_xg[:,3]
# first_level["LGBM_0"] = getVal[:,0]
# first_level["LGBM_1"] = getVal[:,1]
# first_level["LGBM_2"] = getVal[:,2]
# first_level["LGBM_3"] = getVal[:,3]

In [ ]:
# first_level_test = pd.DataFrame(test_pred_cb, columns = ["catboost_0","catboost_1","catboost_2","catboost_3"])
# first_level_test["XGB_0"] = test_pred_xg[:,0]
# first_level_test["XGB_1"] = test_pred_xg[:,1]
# first_level_test["XGB_2"] = test_pred_xg[:,2]
# first_level_test["XGB_3"] = test_pred_xg[:,3]
# first_level_test["LGBM_0"] = predictions[:,0]
# first_level_test["LGBM_1"] = predictions[:,1]
# first_level_test["LGBM_2"] = predictions[:,2]
# first_level_test["LGBM_3"] = predictions[:,3]

In [ ]:
# np.save("first_level.npy",first_level)
# np.save("Y.npy",Y)
# np.save("first_level_test.npy",first_level_test)

In [ ]:
# first_level_train, first_level_val , y_train, y_val = train_test_split(first_level, Y, test_size = 0.3)

In [ ]:
# params_xgb = {
#     "num_rounds" : [100,200,400],
#     "eta" : [0.05, 0.1,0.15, 0.2, 0.25, 0.30],
#     "learning_rate" : [0.28, 0.23, 0.25, 0.2,0.1],
#     "min_child_weight" : [1,3,5,7,10],
#     "gamma" : [0.05, 0.1,0.2, 0.3, 0.4],
#     "colsample_bytree" : [0.5,0.7, 0.8,0.9],
#     "subsample" : [0.8,0.9,0.5,0.7],
#     "max_depth" : [6,7,9,11,13],
#     "n_estimators" : [25,50,100,75],
# }

In [ ]:
# random_search = RandomizedSearchCV(estimator = xgbRegressor, param_distributions = params_xgb, scoring = "f1_weighted", n_jobs = -1, cv = 5, verbose = 3, n_iter=200, )

In [ ]:
# random_search.fit(first_level,Y)

In [ ]:
# random_search.best_estimator_

In [ ]:
# random_search.best_score_

In [ ]:
# metamodel = random_search.best_estimator_
# from sklearn.linear_model import RidgeClassifierCV
# # metamodel = RidgeClassifierCV(cv = 5, scoring = "f1_weighted")

In [ ]:
# metamodel.fit(first_level, Y, eval_set = [(first_level_train,y_train),(first_level_val, y_val)], verbose = 20, early_stopping_rounds = 20)
# metamodel.fit(first_level, Y)

In [ ]:
# ensemble_pred = metamodel.predict(first_level_val)
# test_pred = metamodel.predict(first_level_test)

In [ ]:
# print("Val F1-Score :", f1_score(y_val, ensemble_pred, average = "weighted"))

In [ ]:
# model_performance_sc_plot(ensemble_pred, y_val, "Validation")

In [ ]:

# from imblearn.over_sampling import SMOTE
# sm = SMOTE(strategy = "not majority", random_state = 42)
# x_res,y_res = sm.fit_resample(X,Y)
# xgbRegressor.fit(x_res, y_res, eval_set = [(X,Y),(x_val1, y_val1)], verbose = 20, early_stopping_rounds = 120)
# test_pred = xgbRegressor.predict(x_test)


In [ ]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
features = []
for i in train.columns:
    if i in ["Accident_ID" , "Severity"]:
        continue
    features.append(i)
target = ["Severity"]

In [ ]:
train.columns

In [ ]:
df = pd.DataFrame()
train["type"] = "train"
test["type"] = "test"
df = pd.concat([df,train,test],axis = 0)

In [ ]:
cats = ["Days_Since_Inspection", "Total_Safety_Complaints", "Accident_Type_Code", "Violations"]
for cat in cats:
    one_hot = pd.get_dummies(df[cat], prefix = str(cat))
    train.drop(cat, axis = 1, inplace= True)
    test.drop(cat,axis = 1, inplace= True)
    train = train.join(one_hot.loc[df["type"]=="train",:])
    test = test.join(one_hot.loc[df["type"]=="test",:])
    print(i)

In [ ]:
for col in train.columns:
    if col not in test.columns:
        print(col)

In [ ]:
train.drop(["type"],axis = 1, inplace= True)
test.drop(["type"],axis = 1, inplace= True)

In [ ]:
X = train.drop(["Severity"],axis = 1)
Y = train["Severity"]

In [ ]:
x_test = test

In [ ]:
from xgboost import XGBClassifier
xgbRegressor = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.05, gamma=0.3,
              learning_rate=0.23, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=650, n_jobs=-1,
              nthread=None, num_rounds=200, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=0.8, verbosity=1)

In [ ]:
for col in X.columns:
    if col not in x_test:
        print(col)

In [ ]:
from sklearn.model_selection import KFold
n_splits = 5
kf = KFold(n_splits = n_splits, shuffle = True)
y_pred_xg = np.zeros((Y.shape[0],4))
sums_score = 0
test_pred_xg = np.zeros((test.shape[0], 4))
for train_index, test_index in kf.split(X):
    x_train, y_train = X.loc[train_index,:] , Y[train_index]
    x_val, y_val = X.loc[test_index,:] , Y[test_index]
    xgbRegressor.fit(x_train , y_train, verbose = 100)
    y_pred_x = xgbRegressor.predict_proba(x_val)
    print(y_pred_x.shape)
    y_pred_xg[test_index] += y_pred_x.reshape(-1,4)/n_splits
    y_pred_x = xgbRegressor.predict(x_val)
    test_pred_xg += xgbRegressor.predict_proba(x_test)/n_splits
    print("F1_Score val1 : ", f1_score(y_train, xgbRegressor.predict(x_train),average='weighted'))
    print("F1_Score val1 : ", f1_score(y_val, y_pred_x,average='weighted'))
    sums_score +=  f1_score(y_val, y_pred_x,average='weighted')
print("CV_mean: ", sums_score / n_splits)

In [ ]:
xgbRegressor.fit(X,Y)
test_pred = xgbRegressor.predict(x_test)

In [ ]:
y_pred = test_pred
y_pred = y_pred.astype(np.int32)
y_pred = lb.inverse_transform(y_pred)

In [ ]:
ids = []
for i in range(test.shape[0]):
    ids.append(test.loc[i,"Accident_ID"])

In [ ]:
sub = pd.DataFrame(ids, columns= ["Accident_ID"])

In [ ]:
sub["Severity"] = y_pred

In [ ]:
sub.head()

In [ ]:
sub.to_csv("Submission.csv",index = False)